In [13]:
import random
from collections import defaultdict
import argparse
import pandas as pd
from tqdm import tqdm

N=10
step=3
data_file="../data/TopN-jupyter.txt"

In [14]:
data_fields = ['user_id', 'item_id']
data = pd.read_table(data_file, names=data_fields)
# 二维字典
train_data = {}

# 按照1:9划分数据集
for (user, item) in data.itertuples(index=False):
    train_data.setdefault(user,{})
    train_data[user][item] = 1

n_users = len(set(data['user_id'].values))
n_items = len(set(data['item_id'].values))
train_data

{'A': {'b': 1, 'd': 1},
 'B': {'a': 1, 'b': 1, 'c': 1},
 'C': {'a': 1, 'b': 1, 'd': 1},
 'D': {'a': 1, 'e': 1}}

In [15]:
# 建立item_user倒排表
item_users = dict()
for u, items in train_data.items():
    for i in items:
        if i not in item_users:
            item_users[i] = set()
        item_users[i].add(u)
item_users

{'b': {'A', 'B', 'C'},
 'd': {'A', 'C'},
 'a': {'B', 'C', 'D'},
 'c': {'B'},
 'e': {'D'}}

In [16]:
user_item = train_data
rec_item = defaultdict(list)
for u in tqdm(user_item):
    # 每个物品第几步被扩散到
    visit_step = defaultdict(int)
    # 物品扩散到的次数
    visit_cnt = defaultdict(int)

    user_set = set()
    user_set.add(u)
    item_set = set()
    for k in range(step):
        if k % 2 == 0:
            # 用户扩散项目
            for user in user_set:
                for item in user_item[user]:
                    if item not in visit_step:
                        visit_step[item] = k + 1
                    visit_cnt[item] += 1
                item_set.update(set(user_item[user]))
            user_set = set()
        else:
            # 项目扩散用户
            for item in item_set:
                user_set.update(set(item_users[item]))
            item_set = set()

    res = ((pd.DataFrame(visit_cnt, index=[0]).append(pd.DataFrame(visit_step, index=[1]))).T).sort_values(
        [1, 0], ascending=[1, 0]).index.tolist()

    rec_item[u] = [i for i in res if i not in user_item[u]]
rec_item 

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 286.47it/s]


defaultdict(list,
            {'A': ['a', 'c'],
             'B': ['d', 'e'],
             'C': ['e', 'c'],
             'D': ['b', 'c', 'd']})